In [ ]:
'''
Main program
'''
from pathlib import Path
import os, shutil
import zipfile
import tarfile
import glob

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import xarray as xr
import pandas as pd
import logging

from holoviews import streams
import holoviews as hv
import panel as pn
import param
hv.extension('bokeh')
from holoviews import opts

import cartopy.crs as ccrs

from s2driver import driver_S2_SAFE as S2

from grs import Product, AuxData, acutils, CamsProduct, L2aProduct, __version__

import grstbx
from grstbx import visual

pn.extension()


opj = os.path.join
__version__

### Indicate the path where you put the look-up table file

In [ ]:
lut_file = '/data/vrtc/xlut/toa_lut_opac_wind_light.nc'
lut_file = '/data/vrtc/xlut/toa_lut_opac_wind_light_v2.nc'
trans_lut_file = '/data/vrtc/xlut/transmittance_lut_opac_wind_light_v2.nc'

### Set the path of your Sentinel-2 image and corresponding CAMS file

In [ ]:
workdir = '/data/satellite/Sentinel-2/projet/robert/L1C/2021/'
files = pn.widgets.FileSelector(workdir)

files

In [ ]:
#file = files.value[0]
#file = '/data/satellite/Sentinel-2/L1C/45TVG/2022/06/12/S2B_MSIL1C_20220612T050659_N0400_R019_T45TVG_20220612T070737.SAFE'
file = '/data/satellite/Sentinel-2/L1C/45TWG/2022/06/12/S2B_MSIL1C_20220612T050659_N0400_R019_T45TWG_20220612T070737.SAFE'

tile = file.split('_')[-2][1:]
dem_file = '/home/harmel/Dropbox/Dropbox/satellite/dem/COP-DEM_GLO-30-DGED_'+tile+'.tif'

cams_file = '/data/cams/world/cams_forecast_2021-01.nc'

file_nc = file.replace('.SAFE', '.nc')

### First, check the bands available:

In [ ]:
S2.INFO

In [ ]:
bandIds = range(13)
resolution = 20

### Load the image and convert it into netcdf for further fatest loading (not necessary)

In [ ]:

prod = Product(xr.open_dataset(file_nc))


In [ ]:
prod.raster

In [ ]:
import cycler

n = 13
color = plt.cm.Spectral_r(np.linspace(0, 1,n))
mpl.rcParams['axes.prop_cycle'] = cycler.cycler('color', color)
prod.raster.SRF.plot(x='wl_hr',hue='wl',lw=1,add_legend=False)

Check RGB image

In [ ]:
dem=xr.open_dataset(dem_file).squeeze()
dem=dem.rename_vars({'band_data':'elevation'})

x, y = np.gradient(dem.elevation)
azi=prod.raster.mean_solar_azimuth
sza=prod.raster.mean_solar_zenith_angle
azir=np.radians(azi%360)
szar=np.radians(sza)

slope =np.arctan(np.sqrt(x * x + y * y))
aspect = np.arctan2(-x,y)

shaded = np.cos(szar) * np.cos(slope) + np.sin(szar) * np.sin(slope) * np.cos(azir - aspect)

dem=dem.merge(xr.Dataset(dict(shaded=(["y", "x"],shaded),
                              slope=(["y", "x"],slope)),
                         
                       
                  coords=dict(x=dem.x,
                              y=dem.y),
               ))

In [ ]:
prod.raster

In [ ]:
str_epsg = str(prod.raster.rio.crs)
zone = str_epsg[-2:]
is_south = str_epsg[2] == 7
proj = ccrs.UTM(zone, is_south)

plt.figure(figsize=(15,15))
prod.raster.bands.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,subplot_kws=dict(projection=proj))

In [ ]:
prod.raster.sza.mean()

In [ ]:
plt.figure(figsize=(15,15))
dem.slope.plot.imshow(robust=True,subplot_kws=dict(projection=proj),cmap=plt.cm.Greys_r,cbar_kwargs={'shrink':0.35})

In [ ]:
plt.figure(figsize=(15,15))
dem.shaded.plot.imshow(robust=True,subplot_kws=dict(projection=proj),cmap=plt.cm.Greys_r,cbar_kwargs={'shrink':0.35})

### Visualize, interact and subset your region of interest

In [ ]:
v=visual.view_spectral(prod.raster.bands,reproject=True)

In [ ]:
v.title="## S2 L1C"
v.minmaxvalues=(0,0.3)
v.minmax=[0,0.5]
v.visu()

In [ ]:
geom_ = v.get_geom(v.aoi_stream,crs=prod.raster.rio.crs)

raster_clipped = xr.Dataset()
prod.raster.bands.rio.clip(geom_.geometry.values)
for param in prod.raster.keys():
    da = prod.raster[param]
    if 'x' in da.dims and 'y' in da.dims:
        raster_clipped[param]=da.rio.clip(geom_.geometry.values)
    else:
        raster_clipped[param]=da
raster_clipped.attrs = prod.raster.attrs
prod.raster = raster_clipped

In [ ]:

#%matplotlib ipympl
import cartopy.crs as ccrs
str_epsg = str(raster_clipped.rio.crs)
zone = str_epsg[-2:]
is_south = str_epsg[2] == 7
proj = ccrs.UTM(zone, is_south)

plt.figure(figsize=(15,15))
raster_clipped.bands.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,subplot_kws=dict(projection=proj))

## Start GRS processing

If you want to process the whole tile comment the following line

In [ ]:
prod.raster = raster_clipped

Start by checking the CAMS data

In [ ]:
##################################
# GET ANCILLARY DATA (Pressure, O3, water vapor, NO2...
##################################

cams = cams_product(prod.raster, cams_file=cams_file)
#cams.wls=[355,380,400,440,469,500, 550, 645,670,800, 865,1020, 1240,1640,2130]
cams.load()

In [ ]:
prod.date_str

In [ ]:
date = prod.date.strftime('%Y-%m-%d')
cams_ = xr.open_dataset(cams_file, decode_cf=True,
                               chunks={'time': 1, 'x': 500, 'y': 500})
cams_ = cams_.sel(time=date)

In [ ]:
self=prod
xmin, ymin, xmax, ymax = prod.raster.rio.bounds()
lonmin, latmin, lonmax, latmax = self.lonmin, self.latmin, self.lonmax, self.latmax

cams_ = cams_.sel(latitude=slice(latmax + 1, latmin - 1))
# check if image is on Greenwich meridian and adapt longitude convention
if cams_.longitude.min()>=0:
    if lonmin <= 0 and lonmax >= 0:

            cams_ = cams_.assign_coords({"longitude": (((cams_.longitude + 180) % 360) - 180)}).sortby('longitude')
    else:
        # set longitude between 0 and 360 deg
        lonmin, lonmax, = lonmin % 360, lonmax % 360

# slicing
cams_ = cams_.sel(longitude=slice(lonmin - 1, lonmax + 1)).compute()

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(16,6),sharey=True,sharex=True)

aod550.mean('time').plot.imshow(cmap=plt.cm.Spectral_r,robust=True,cbar_kwargs={'shrink':0.65},ax=axs[0])
aod550.std('time').plot.imshow(cmap=plt.cm.Spectral_r,robust=True,cbar_kwargs={'shrink':0.65},ax=axs[1])


In [ ]:
plt.figure(figsize=(15,15))
fig,axs = plt.subplots(1,2,figsize=(16,6),sharey=True,sharex=True)
aod550.sel(time='2022-06-12T06').plot.imshow(ax=axs[0],cmap=plt.cm.Spectral_r,vmin=0.28,vmax=0.51,cbar_kwargs={'shrink':0.65})
aod550.sel(time='2022-06-12T12').plot.imshow(ax=axs[1],cmap=plt.cm.Spectral_r,vmin=0.28,vmax=0.51,cbar_kwargs={'shrink':0.65})

In [ ]:
plt.figure(figsize=(15,12))

aod550.sel(time='2022-06-12').mean('time').plot.imshow(cmap=plt.cm.Spectral_r,vmin=0.28,vmax=0.51,cbar_kwargs={'shrink':0.65})


In [ ]:
params=['aod550','amaod550', 'bcaod550', 'duaod550', 'niaod550', 'omaod550', 'ssaod550', 'suaod550']
Nrows = (len(params) + 4) // 4
fig, axs = plt.subplots(Nrows, 4, figsize=(4 * 4.2, Nrows * 3.5))
axs = axs.ravel()
[axi.set_axis_off() for axi in axs]
aod550 = cams.raster['aod550']
aod550
for i, param in enumerate(params):
    if i == 0:
        fig = cams.raster[param].plot.imshow(robust=True, ax=axs[i],cmap=plt.cm.Spectral_r)
        fig.axes.set_title(param)
    else:
        fig = (cams.raster[param]/aod550).plot.imshow(robust=True, ax=axs[i],cmap=plt.cm.Spectral_r)
        fig.axes.set_title('prop. of '+param)
    #fig.colorbar.set_label(self.raster[param].units)
    fig.axes.set(xticks=[], yticks=[])
    fig.axes.set_ylabel('')
    fig.axes.set_xlabel('')
plt.tight_layout()

In [ ]:
#cams.plot_params(params=['aod550', 'aod2130', 'ssa550', 't2m', 'msl', 'sp','tcco', 'tchcho', 'tc_oh', 'tc_ch4', 'tcno2', 'gtco3', 'tc_c3h8', 'tcwv', 'u10', 'v10'], cmap=plt.cm.Spectral_r)
#'ammonium_aerosol_optical_depth_550nm', 'black_carbon_aerosol_optical_depth_550nm',
#                'dust_aerosol_optical_depth_550nm',
#                'nitrate_aerosol_optical_depth_550nm', 'organic_matter_aerosol_optical_depth_550nm',
#                'sea_salt_aerosol_optical_depth_550nm',
#                'sulphate_aerosol_optical_depth_550nm',
cams.plot_params(params=['v10','u10', 'msl', 'sp','t2m', 'tcco', 'tc_ch4', 'tcno2', 'gtco3', 'tcwv',
 'amaod550', 'bcaod550', 'duaod550', 'niaod550', 'omaod550', 'ssaod550', 'suaod550',
 'aod1240', 'aod469', 'aod550', 'aod670', 'aod865',
 
 ],


                 cmap=plt.cm.Spectral_r)
plt.show()

In [ ]:
cams.raster

In [ ]:
cams.plot_params(params=['aod550', 'aod2130', 'ssa550',
                                   't2m', 'msl', 'sp','tcco', 'tchcho',
                                   'tc_oh', 'tc_ch4', 'tcno2', 'gtco3',
                                   'tc_c3h8', 'tcwv', 'u10', 'v10'],
                 cmap=plt.cm.Spectral_r)
plt.show()

You may also want to visualize the module wind speed or the mean square slope (sigma2 taken from the Cox Munk isotropic model)

In [ ]:

wind = np.sqrt(cams.raster['v10']**2+cams.raster['u10']**2)
sigma2=(wind+0.586)/195.3

fig,axs = plt.subplots(1,2,figsize=(15,6),sharey=True,sharex=True)
wind.plot.imshow(ax=axs[0],cmap=plt.cm.Spectral_r)
sigma2.plot.imshow(ax=axs[1],cmap=plt.cm.Spectral_r)

You can also plot a cams parameter as a new layer above the S2 RGB image

In [ ]:
plt.figure(figsize=(15,15))

prod.raster.bands.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,subplot_kws=dict(projection=proj))
wind.plot.imshow( cmap=plt.cm.Spectral_r,robust=True,alpha=0.5,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35})

Take the mean values for LUT selection and interpolation 

In [ ]:
_sigma2 = sigma2.mean().values
_wind = wind.mean().values
print(_sigma2,_wind)

Prepare spectral bands for further processing andload LUT

In [ ]:
#####################################
# SUBSET RASTER TO KEEP REQUESTED BANDS
#####################################
if prod.bcirrus:
    prod.cirrus = prod.raster.bands.sel(wl=prod.bcirrus, method='nearest')
if prod.bwv:
    prod.wv = prod.raster.bands.sel(wl=prod.bwv, method='nearest')
prod.raster = prod.raster.sel(wl=prod.wl_process, method='nearest')


#####################################
# LOAD LUT FOR ATMOSPHERIC CORRECTION
#####################################
#logging.info('loading lut...' + prod.lutfine)

Ttot_Ed = xr.open_dataset(trans_lut_file)
Ttot_Ed['wl'] = Ttot_Ed['wl'] * 1000

aero_lut = xr.open_dataset(lut_file)
aero_lut['wl']=aero_lut['wl']*1000
aero_lut['aot'] = aero_lut.aot.isel(wind=0).squeeze()
# remove URBAN aerosol model for this example.drop_sel(model='URBA_rh70')
models=aero_lut.model.values
#aero_lut
wl_true = prod.raster.wl_true
_auxdata = auxdata(wl=wl_true)#wl=masked.wl)
sunglint_eps = _auxdata.sunglint_eps#['mean'].interp(wl=wl_true)
rot = _auxdata.rot#.interp(wl=wl_true)

## Set aerosol model from CAMS data

Get spectral aerosol optical thickness from CAMS raster

In [ ]:
cams_aot_mean = cams.cams_aod.mean(['x','y'])
cams_aot_ref = cams.cams_aod.interp(wl=550,method='quadratic').compute()
cams_aot_ref_mean =  cams_aot_ref.mean(['x','y'])


Check proximity with tabulated models (LUT) and select representative aerosol model

In [ ]:


fig, axs = plt.subplots(1, 2, figsize=(18, 4.5))
aero_lut.aot.sel(model=models,aot_ref=1).plot(ax=axs[0],hue='model')#,add_legend=False)
(cams_aot_mean/cams_aot_ref_mean).plot(ax=axs[0],color='black')
aero_lut.aot.isel(model=[4,2]).mean('model').sel(aot_ref=1).plot(ax=axs[0],color='grey')
lut_aod=aero_lut.aot.sel(model=models,aot_ref=1).interp(wl=cams.cams_aod.wl)
rank = np.abs((cams_aot_mean/cams_aot_ref_mean)-lut_aod).sum('wl') 
axs[1].bar(x=rank.model, height=rank.values)
plt.xticks(rotation=30, ha='right')
plt.show()

idx = np.abs((cams_aot_mean/cams_aot_ref_mean)-lut_aod).sum('wl').argmin()
opac_model = aero_lut.sel(model=models).model.values[idx]
print(opac_model)

#    absorbing gases correction


In [ ]:

gas_trans = acutils.gaseous_transmittance(prod, cams)
Tg_raster = gas_trans.get_gaseous_transmittance()
#Tg_raster

In [ ]:
logging.info('correct for gaseous absorption')
for wl in prod.raster.wl.values:
    Tg_ = Tg_raster.sel(wl=wl).interp(x=prod.raster.x, y=prod.raster.y)
    prod.raster['bands'].loc[wl] = prod.raster.bands.sel(wl=wl) / Tg_
    del Tg_
prod.raster.bands.attrs['gas_absorption_correction'] = True

In [ ]:
import gc
gc.collect()

You can check the gaseous transmittance for each spectral band

In [ ]:
plt.figure()
Tg_raster.mean('x').mean('y').plot()
fig,axs = plt.subplots(3,4,figsize=(15,9),sharey=True,sharex=True)
axs=axs.ravel()
[axi.set_axis_off() for axi in axs]
for iwl in range(len(prod.wl_process)):
    #axs[iwl].set_axis_on()
    Tg_raster.isel(wl=iwl).plot(ax=axs[iwl],cmap=plt.cm.Spectral_r)
    axs[iwl].set_title( str(Tg_raster.isel(wl=iwl).wl.values))


# Water mask


In [ ]:
# Compute NDWI

#green = prod.raster.bands.sel(wl=565,method='nearest')
#nir = prod.raster.bands.sel(wl=prod.b865)

green = prod.raster.bands.sel(wl=prod.bvis,method='nearest')
nir = prod.raster.bands.sel(wl=prod.bnir,method='nearest')#prod.b865)
swir = prod.raster.bands.sel(wl=prod.bswir)
b2200 = prod.raster.bands.sel(wl=prod.bswir2)

ndwi = (green - nir) / (green + nir)
ndwi_swir = (green - swir) / (green + swir)

prod.raster['ndwi'] = ndwi
prod.raster.ndwi.attrs = {
'description': 'Normalized difference spectral index between bands at ' + str(prod.bvis) + ' and ' + str(
    prod.bnir) + ' nm', 'units': '-'}
prod.raster['ndwi_swir'] = ndwi_swir
prod.raster.ndwi_swir.attrs = {
'description': 'Normalized difference spectral index between bands at ' + str(prod.bvis) + ' and ' + str(
    prod.bswir) + ' nm', 'units': '-'}

In [ ]:
fig = plt.figure(figsize=(25, 15))
ax = plt.subplot(1, 2, 1, projection=proj)
ndwi.plot.imshow(robust=True,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35})#,vmin=-0.1,vmax=0.1,cmap=plt.cm.RdBu_r) 
ax = plt.subplot(1, 2, 2, projection=proj)
ndwi_swir.plot.imshow(robust=True,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35})#,vmin=-0.1,vmax=0.1,cmap=plt.cm.RdBu_r) 

In [ ]:
mask =  (ndwi_swir > 0) &  (b2200 < 0.2)#(ndwi > -0.0) &
masked = prod.raster.bands.where(mask)

from matplotlib.colors import ListedColormap
# binary cmap
bcmap = ListedColormap(['khaki', 'lightblue'])
xmask = xr.where(mask,1,0)

fig = plt.figure(figsize=(25, 15))
ax = plt.subplot(1, 2, 1, projection=proj)
xmask.plot.imshow(ax=ax,cmap=bcmap, cbar_kwargs={'ticks': [0, 1], 'shrink': 0.4})
ax = plt.subplot(1, 2, 2, projection=proj)
prod.raster['raa'].sel(wl=560).plot.imshow(cmap=plt.cm.Spectral_r, ax=ax, robust=True,cbar_kwargs={'shrink':0.4})
masked.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,ax=ax)


In [ ]:
plt.figure(figsize=(15,15))
(dem.shaded+1).plot.imshow(robust=True,subplot_kws=dict(projection=proj),cmap=plt.cm.Greys_r,cbar_kwargs={'shrink':0.4})
masked.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,subplot_kws=dict(projection=proj))

Check pressure

In [ ]:
presure_msl = cams.raster.msl.interp(y=prod.raster.y,x=prod.raster.x)
palt = presure_msl* (1. - 0.0065 * dem.elevation/ 288.15) ** 5.255

In [ ]:
plt.figure(figsize=(15,15))
palt.plot.imshow(robust=True,subplot_kws=dict(projection=proj),cmap=plt.cm.Spectral_r,cbar_kwargs={'shrink':0.65})

If you are happy with your mask you continue and proceed with masking, tweak the threshold values again, otherwise.

In [ ]:
prod.raster['bands'] = masked
prod.raster['sza'] =  prod.raster['sza'].where(mask)
                                               

Preparation of LUT and other input parameters

In [ ]:
aero_lut.model

In [ ]:
models = ['ARCT_rh70', 'COAV_rh70', 'DESE_rh70', 'MACL_rh70', 'URBA_rh70']
Nwl,height,width  = prod.raster.bands.shape
chunk = 256
pressure_ref=101500.

cams_aot_ref = cams.cams_aod.interp(wl=550,method='quadratic').compute() #*0.7#.interp(x=prod.raster.x,y=prod.raster.y).compute()#.plot.imshow()
aot_ref_raster = cams_aot_ref
iwl_swir = [-2, -1]
aero_lut_=aero_lut.sel(wind=_wind,method='nearest').sel(model=models[0])#.isel(model=[4,2]).mean('model') # #opac_model)

In [ ]:
aero_lut_

In [ ]:
ang_resol={'sza':0.1, 'vza':0.1, 'raa_round':0}
szamin,szamax=float(prod.raster['sza'].min()),float(prod.raster['sza'].max())
vzamin,vzamax=float(prod.raster['vza'].isel(wl=0).min()),float(prod.raster['vza'].isel(wl=0).max())
# check for out-of-range
def check_out_of_range(vmin,vmax,ceiling=88):
    vmin = np.max([0,vmin])
    vmax = np.min([ceiling,vmax])
    return vmin,vmax
szamin,szamax = check_out_of_range(szamin,szamax)
vzamin,vzamax = check_out_of_range(vzamin,vzamax,ceiling=25)

sza_ = np.arange(szamin,szamax+ang_resol['sza'],ang_resol['sza'])
vza_ = np.arange(vzamin,vzamax+ang_resol['vza'],ang_resol['vza'])

azi_=(180-np.unique(prod.raster['raa'].isel(wl=0).round(ang_resol['raa_round'])))%360
azi_ = azi_[~np.isnan(azi_)]
#azi_ = np.arange(0,361,1)
print(szamin,szamax)
print(vzamin,vzamax)
print(azi_)

In [ ]:
sza_lut_step=2
vza_lut_step=2
sza_slice=slice(np.min(sza_)-sza_lut_step,np.max(sza_)+sza_lut_step)
vza_slice=slice(np.min(vza_)-vza_lut_step,np.max(vza_)+vza_lut_step)
tweak=3
aot_ref_ =np.unique((aot_ref_raster/tweak).round(3))*tweak
aot_ref_min = aot_ref_raster.min()
aot_ref_max = aot_ref_raster.max()
aot_lut = aero_lut_.aot.interp(wl=wl_true,method='quadratic')
aot_lut = aot_lut.interp(aot_ref=np.linspace(aot_ref_min,aot_ref_max,1000))#.plot(hue='wl')

In [ ]:
Rdiff_lut = aero_lut_.I.sel(sza=sza_slice,vza=vza_slice).interp(wl=wl_true,method='quadratic').interp(azi=azi_)

In [ ]:
Rdiff_lut =  Rdiff_lut.interp(sza=sza_,vza=vza_)
Rray =  Rdiff_lut.sel(aot_ref=0)
Rdiff_lut = Rdiff_lut.interp(aot_ref=aot_ref_,method='quadratic')

szas = Rdiff_lut.sza.values
vzas = Rdiff_lut.vza.values
azis = Rdiff_lut.azi.values
aot_refs = Rdiff_lut.aot_ref.values

In [ ]:
Ttot_Ed_ = Ttot_Ed.sel(model=opac_model).sel(wind=_wind, method='nearest').interp(sza=szas).interp(
    aot_ref=aot_ref_, method='quadratic').interp(wl=wl_true, method='cubic').Ttot_Ed
Ttot_Lu_ = Ttot_Ed.sel(model=opac_model).sel(wind=_wind, method='nearest').interp(sza=vzas).interp(
    aot_ref=aot_ref_, method='quadratic').interp(wl=wl_true, method='cubic').Ttot_Ed ** 1.05

_sunglint_eps=sunglint_eps.values
# prepare aerosol parameters
aot_ref_raster = cams_aot_ref.interp(x= prod.raster.x, y= prod.raster.y).drop('wl')
aot_ref_raster.name='aot550'
#_aot = aot_lut.interp(aot_ref=_aot_ref)
#_pressure = cams.raster.sp.interp(x= prod.raster.x, y= prod.raster.y).values
_pressure = palt.values
_rot = rot.values

### And finally run the GRS kernel !!!

Rrs_tmp = np.full((Nwl, height, width), np.nan, dtype=prod._type)
Rf_tmp = np.full((height,width), np.nan, dtype=prod._type)
for iy in range(0, height, chunk):
    yc = min(height, iy + chunk)
    
    for ix in range(0, width, chunk):
        xc = min(width, ix + chunk)
       
        _band_rad = prod.raster.bands[:, iy:yc, ix:xc]
        
        Nwl, Ny, Nx = _band_rad.shape
        #if Ny == 0 or Nx == 0:
        #    continue
        arr_tmp = np.full((Nwl, Ny, Nx), np.nan, dtype=np.float32)
        
        # subsetting
        _sza = prod.raster.sza[iy:yc, ix:xc]  # .values
        _raa = prod.raster.raa[:, iy:yc, ix:xc]
        _azi = (180. - _raa) % 360
        _vza = prod.raster.vza[:, iy:yc, ix:xc]
        _vza_mean = np.mean(_vza, axis=0).values
        _air_mass_ = acutils.misc.air_mass(_sza, _vza).values  # air_mass[:, iy:yc,ix:xc] #air_mass(_sza,_vza).values #_p_slope = prod.raster.p_slope[:, iy:yc,ix:xc]
        _p_slope_ = prod.p_slope(_sza, _vza, _raa, sigma2=_sigma2).values  # _p_slope[:, iy:yc,ix:xc]
        _aot_ref = aot_ref_raster.values[iy:yc, ix:xc]
        _pressure_ = _pressure[iy:yc, ix:xc] / pressure_ref           
        
        # get LUT values
        _Rdiff = acutils._interp_Rlut(szas, _sza.values,
                                  vzas, _vza.values,
                                  azis, _azi.values,
                                  aot_refs, _aot_ref,
                                  Nwl, Ny, Nx, Rdiff_lut.values)
        _Rray = acutils._interp_Rlut_rayleigh(szas, _sza.values,
                                  vzas, _vza.values,
                                  azis, _azi.values,
                                  Nwl, Ny, Nx, Rray.values)
        _Rdiff = _Rdiff + (_pressure_ - 1) *  _Rray 
       
        _aot = acutils._interp_aotlut(aot_lut.aot_ref.values, _aot_ref, Nwl, Ny, Nx, aot_lut.values)
        
        #  correction for diffuse light
        Rcorr = _band_rad.values - _Rdiff
        
         # construct wl,y,x raster for Rayleigh optical thickness
        _rot_raster = acutils._multiplicate(_rot, _pressure_, arr_tmp)
       
        # direct transmittance up/down
        Tdir = acutils.misc.transmittance_dir(_aot, _air_mass_, _rot_raster)

        # vTotal transmittance (for Ed and Lu)
        Tdown = acutils._interp_Tlut(szas, _sza.values, Ttot_Ed_.aot_ref.values, _aot_ref, Nwl, Ny, Nx,
                                         Ttot_Ed_.values)
        Tup = acutils._interp_Tlut(vzas, _vza_mean, Ttot_Ed_.aot_ref.values, _aot_ref, Nwl, Ny, Nx, Ttot_Lu_.values)
        Ttot_du = Tdown * Tup
        
        Rf = np.full((len(iwl_swir), Ny, Nx), np.nan, dtype=np.float32)
        
        for iwl in iwl_swir:
            Rf[iwl] = Rcorr[iwl] / (Tdir[iwl] * _sunglint_eps[iwl] * _p_slope_[iwl])
        Rf[Rf<0]=0.
        Rf = np.min(Rf, axis=0)
        Rf_tmp[iy:yc, ix:xc] = Rf
        
        Rf = acutils._multiplicate(_sunglint_eps, Rf, arr_tmp)
        Rf = Tdir * _p_slope_ * Rf
        
        Rrs_tmp[:, iy:yc, ix:xc] = ((Rcorr - Rf) / np.pi)/ Ttot_du
print('success')

### If you have sufficient CPUs and memory, you try the parallelized multiprocessor method

In [ ]:
from multiprocessing import Pool  # Process pool
from multiprocessing import sharedctypes
import itertools
global chunk_process, pool
pool = None
type= np.float32
Nproc=8
Rrs_result = np.ctypeslib.as_ctypes(np.full((Nwl, height, width), np.nan, dtype=prod._type))
Rf_result = np.ctypeslib.as_ctypes(np.full((height,width), np.nan, dtype=prod._type))
shared_Rrs = sharedctypes.RawArray(Rrs_result._type_, Rrs_result)
shared_Rf = sharedctypes.RawArray(Rf_result._type_, Rf_result)

In [ ]:

def chunk_process(args):
    iy, ix = args
    yc = min(height, iy + chunk)
    xc = min(width, ix + chunk)
    Rrs_tmp = np.ctypeslib.as_array(shared_Rrs)
    Rf_tmp = np.ctypeslib.as_array(shared_Rf)
    
    _band_rad = prod.raster.bands[:, iy:yc, ix:xc]

    Nwl, Ny, Nx = _band_rad.shape
    if Ny == 0 or Nx == 0:
        return
    arr_tmp = np.full((Nwl, Ny, Nx), np.nan, dtype=np.float32)

    # subsetting
    _sza = prod.raster.sza[iy:yc, ix:xc]  # .values
    _raa = prod.raster.raa[:, iy:yc, ix:xc]
    _azi = (180. - _raa) % 360
    _vza = prod.raster.vza[:, iy:yc, ix:xc]
    _vza_mean = np.mean(_vza, axis=0).values
    _air_mass_ = acutils.misc.air_mass(_sza, _vza).values  # air_mass[:, iy:yc,ix:xc] #air_mass(_sza,_vza).values #_p_slope = prod.raster.p_slope[:, iy:yc,ix:xc]
    _p_slope_ = prod.p_slope(_sza, _vza, _raa, sigma2=_sigma2).values  # _p_slope[:, iy:yc,ix:xc]
    _aot_ref = aot_ref_raster.values[iy:yc, ix:xc]
    _pressure_ = _pressure[iy:yc, ix:xc] / pressure_ref

    # construct wl,y,x raster for Rayleigh optical thickness
    _rot_raster = acutils._multiplicate(_rot, _pressure_, arr_tmp)

    # get LUT values
    _Rdiff = acutils._interp_Rlut(szas, _sza.values,
                                  vzas, _vza.values,
                                  azis, _azi.values,
                                  aot_refs, _aot_ref,
                                  Nwl, Ny, Nx, Rdiff_lut.values)
    
    _Rray = acutils._interp_Rlut_rayleigh(szas, _sza.values,
                                  vzas, _vza.values,
                                  azis, _azi.values,
                                  Nwl, Ny, Nx, Rray.values)
    _Rdiff = _Rdiff + (_pressure_ - 1) *  _Rray 
       
    
    _aot = acutils._interp_aotlut(aot_lut.aot_ref.values, _aot_ref, Nwl, Ny, Nx, aot_lut.values)

    #  correction for diffuse light
    Rcorr = _band_rad.values - _Rdiff
    
    # direct transmittance up/down
    Tdir = acutils.misc.transmittance_dir(_aot, _air_mass_, _rot_raster)

    # vTotal transmittance (for Ed and Lu)
    Tdown = acutils._interp_Tlut(szas, _sza.values, Ttot_Ed_.aot_ref.values, _aot_ref, Nwl, Ny, Nx,
                                 Ttot_Ed_.values)
    Tup = acutils._interp_Tlut(vzas, _vza_mean, Ttot_Ed_.aot_ref.values, _aot_ref, Nwl, Ny, Nx, Ttot_Lu_.values)
    Ttot_du = Tdown * Tup

    Rf = np.full((len(iwl_swir), Ny, Nx), np.nan, dtype=np.float32)

    for iwl in iwl_swir:
        Rf[iwl] = Rcorr[iwl] / (Tdir[iwl] * _sunglint_eps[iwl] * _p_slope_[iwl])
    Rf[Rf<0]=0.
    Rf = np.min(Rf, axis=0)
    Rf_tmp[iy:yc, ix:xc] = Rf
    
    Rf = acutils._multiplicate(_sunglint_eps, Rf, arr_tmp)
    Rf = Tdir * _p_slope_ * Rf
        
    Rrs_tmp[:, iy:yc, ix:xc] = ((Rcorr - Rf) / np.pi)/ Ttot_du
    return

window_idxs = [(i, j) for i, j in
               itertools.product(range(0, height, chunk),
                                 range(0, width, chunk))]

pool = Pool(Nproc)
global pool
res = pool.map(chunk_process, window_idxs)
pool.terminate()
pool = None
print('success')

In [ ]:
xres = xr.Dataset(dict(Rrs=(['wl',"y", "x"],np.ctypeslib.as_array(shared_Rrs)),
                       BRDFg= (["y", "x"],np.ctypeslib.as_array(shared_Rf))),
                  coords=dict(wl=prod.raster.wl,
                              x=prod.raster.x,
                              y=prod.raster.y),
                              )

l2_prod = xr.merge([aot_ref_raster.drop('time'), xres])
l2a = l2a_product(prod, l2_prod, cams, gas_trans)

In [ ]:
vp=visual.view_param(l2a.l2_prod,reproject=True)

In [ ]:
vp.minmax=[0.,0.1]
vp.visu()

# L2A to L2B

### Mask bad quality pixels

In [ ]:
Rrs_blue_avg=0.05
Rrs = l2a.l2_prod.Rrs
mask=xr.where((Rrs.sel(wl=490,method='nearest')<0)|(Rrs.sel(wl=565,method='nearest')<0),1,0)
mask=mask.where((Rrs.sel(wl=443,method='nearest')<2*Rrs_blue_avg),2)
mask=mask.where((Rrs.sel(wl=443,method='nearest')<3*Rrs_blue_avg),3)
mask=mask.where((l2a.l2_prod.wv_band)<0.1,4)


plt.figure(figsize=(15,15))

mask.plot.imshow(cmap=plt.cm.Spectral_r,robust=True,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35})


Apply mask

In [ ]:
l2a.l2_prod['Rrs']=Rrs.where(mask==0)

In [ ]:
plt.figure(figsize=(15,15))
(dem.shaded+1).plot.imshow(robust=True,subplot_kws=dict(projection=proj),cmap=plt.cm.Greys_r,cbar_kwargs={'shrink':0.4})
l2a.l2_prod.Rrs.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,subplot_kws=dict(projection=proj))

## Check Rrs spectra 

In [ ]:
v=visual.view_spectral(l2a.l2_prod.Rrs,reproject=True)

In [ ]:
v.minmax=[0,0.1]
v.visu()

In [ ]:
l2a

In [ ]:
geom_ = v.get_geom(v.aoi_stream,crs=prod.raster.rio.crs)

raster_clipped = xr.Dataset()
l2a.l2_prod.rio.clip(geom_.geometry.values)
for param in l2a.l2_prod.keys():
    da = l2a.l2_prod[param]
    if 'x' in da.dims and 'y' in da.dims:
        raster_clipped[param]=da.rio.clip(geom_.geometry.values)
    else:
        raster_clipped[param]=da
raster_clipped.attrs = prod.raster.attrs


In [ ]:
stacked = raster_clipped.Rrs.sel(wl=slice(400,1000)).stack(gridcell=["y", "x"]).dropna('gridcell',thresh=0)


In [ ]:
group_coord ='wl'
stat_coord='gridcell'
stats = xr.Dataset({'median':stacked.groupby(group_coord).median(stat_coord)})
stats['q25'] = stacked.groupby(group_coord).quantile(0.25,dim=stat_coord)
stats['q75'] = stacked.groupby(group_coord).quantile(0.75,dim=stat_coord)
stats['min'] = stacked.groupby(group_coord).min(stat_coord)
stats['max'] = stacked.groupby(group_coord).max(stat_coord)
stats['mean'] = stacked.groupby(group_coord).mean(stat_coord)
stats['std'] = stacked.groupby(group_coord).std(stat_coord)
stats['pix_num'] = stacked.count(stat_coord)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

bands=[3,2,1]
fig, axs = plt.subplots(1,1, figsize=(10, 6))#,sharey=True



date = stats.time.dt.date.values
axins = inset_axes(axs, width="45%", height="60%",
               bbox_to_anchor=(.55, .4, 0.9, 0.9),
               bbox_transform=axs.transAxes, loc=3)


raster_clipped.Rrs.isel(wl=bands).plot.imshow(robust=True,ax=axins)
axins.set_title('')
axins.set_axis_off()
axs.axhline(y=0,color='k',lw=1)
axs.plot(stats.wl,stats['median'],c='k')
axs.plot(stats.wl,stats['mean'],c='red',ls='--')
axs.fill_between(stats.wl, stats['q25'], stats['q75'],alpha=0.3,color='grey')
axs.set_title(date)
plt.show()

## Check water quality parameters (e.g., Chl-a concentration from diverse "algorithms")¶


### Check Optical Water Types (OWT)

In [ ]:
owt_file = '/DATA/projet/vrac/owt/Spyrakos_et_al_2018_OWT_inland_mean_standardised.csv'
owt =pd.read_csv(owt_file,index_col=0).stack().to_xarray()
owt = owt.rename({'level_1':'wl'})
owt['wl']=owt.wl.astype(float)


In [ ]:
import matplotlib.patches as mpatches

owt_info={
        1:dict(color='olivedrab',label='OWT1: Hypereutrophic waters'),
        2:dict(color='black',label='OWT2: Common case waters'),
        3:dict(color='cadetblue',label='OWT3: Clear waters'),
        4:dict(color='tan',label='OWT4: Turbid waters with organic content'),
        5:dict(color='chocolate',label='OWT5: Sediment-laden waters'),
        6:dict(color='teal',label='OWT6: Balanced optical effects at shorter wavelengths'),
        7:dict(color='blueviolet',label='OWT7: Highly productive cyanobacteria-dominated waters'),
        8:dict(color='plum',label='OWT8: Productive with cyanobacteria waters'),
        9:dict(color='red',label='OWT9: OWT2 with higher $R_{rs}$ at shorter wavelengths'),#'slategrey'
        10:dict(color='orange',label='OWT10: CDOM-rich waters'),
        11:dict(color='gold',label='OWT11: CDOM-rich with cyanobacteria waters'),
        12:dict(color='firebrick',label='OWT12: Turbid waters with cyanobacteria'),
        13:dict(color='mediumblue',label='OWT13: Very clear blue waters'),
        }
colors = ['olivedrab','black','cadetblue','tan','chocolate','teal','blueviolet','plum','red','orange','gold','firebrick','mediumblue']
cmap_owt = mpl.colors.ListedColormap(colors)

patch = []
for key,info in owt_info.items():
    patch.append(mpatches.Patch(color=info['color'], label=info['label']))

fig, ax = plt.subplots(nrows=1,ncols=1, sharex=True,figsize=(9, 6))
ax.minorticks_on()
for iowt,group in owt.groupby('owt'):
    
    group.plot(color=owt_info[iowt]['color'],lw=3)
ax.set_title('' )
ax.set_ylabel('$Standardized\ R_{rs}\ (nm^{-1})$',fontsize=20)
ax.set_xlabel('$Wavelength\ (nm)$',fontsize=20)    
plt.legend(handles=patch,fontsize=13,bbox_to_anchor=(1, .5, 0.5, 0.5))

In [ ]:
def SAM(R1,R2):
    denum=(R1*R2).sum('wl')
    denom = (R1**2).sum('wl')**0.5 * (R2**2).sum('wl')**0.5
    return np.arccos(denum/denom)

def SCS(R1,R2):
    R1_avg = R1.mean('wl')
    R2_avg = R2.mean('wl')
    R1_std = R1.std('wl')
    R2_std = R2.std('wl')
    Nwl = len(R1.wl)
    return 1/(Nwl) * ((R1-R1_avg) * (R2-R2_avg)).sum('wl') / (R1_std*R2_std)


Rrs_sat = l2a.l2_prod.Rrs.sel(wl=slice(350,800))#.dropna('wl')

Rrs_owt = owt.interp(wl=Rrs_sat.wl)

owt_sam = SAM(Rrs_sat,Rrs_owt)
owt_scs =SCS(Rrs_sat,Rrs_owt)

owt_delta = owt_scs + (1-2*owt_sam/np.pi)/2

In [ ]:
fig = plt.figure(figsize=(25, 15))
ax = plt.subplot(1, 2, 1, projection=proj)

cmap = plt.get_cmap('tab20c',13)
(owt_delta.fillna(-1).argmax('owt')+1).where(Rrs_sat.isel(wl=1)>0).plot.imshow(vmin=0.5,vmax=13.5,cmap=cmap_owt,cbar_kwargs={ 'ticks':range(1,14),'shrink': 0.4},ax=ax)
ax = plt.subplot(1, 2, 2, projection=proj)
cmap = plt.get_cmap('RdBu')#,13)
(owt_delta.max('owt')).where(Rrs_sat.isel(wl=1)>0).plot.imshow(robust=True,cmap=cmap,ax=ax, cbar_kwargs={ 'shrink': 0.4})

### Total suspended particulate matter (SPM) from Nechad et al., 2010, 2016 formulation
### spm in mg/L

In [ ]:
a = [760, 1.3]
ratio = Rrs.sel(wl=865,method='nearest')/Rrs.sel(wl=565,method='nearest')
spm = a[0] * np.exp(a[1]*ratio)
spm.name='SPM_EA_BR'
spm.attrs['units']='mg/L'
spm.attrs['description']='Concentration of suspended particulate matter from band ratio 865 over 565 nm'
#spm= spm.where((spm >= 0) & (spm <= 50000))
a = [1.378, 4.246]
ratio = Rrs.sel(wl=705,method='nearest')/Rrs.sel(wl=565,method='nearest')
spm2 = a[0] * np.exp(a[1]*ratio)
spm2.name='SPM_EA_BR'
spm2.attrs['units']='mg/L'
spm2.attrs['description']='Concentration of suspended particulate matter from band ratio 705 over 565 nm'

a = [6.384, 3.215,-57.7]
ratio = Rrs.sel(wl=705,method='nearest')/Rrs.sel(wl=565,method='nearest')
spm3 = a[0] * np.exp(a[1]*ratio)+a[2]
spm3.name='SPM_EA_BR'
spm3.attrs['units']='mg/L'
spm3.attrs['description']='Concentration of suspended particulate matter from band ratio 705 over 565 nm'

#spm= spm.where((spm >= 0) & (spm <= 50000))
_spm = 42.03* (Rrs.isel(wl=2)-Rrs.isel(wl=5))**.7+0.41
_spm.name='SPM_P2023'

In [ ]:
plt.figure(figsize=(25,18))
colors = ['mediumblue','cadetblue','teal','gold','orange','chocolate','firebrick']
cmap = mpl.colors.LinearSegmentedColormap.from_list('spm',colors)
fig = plt.figure(figsize=(25, 15))
ax = plt.subplot(1, 3, 1, projection=proj)
spm.plot.imshow(cmap=cmap,ax=ax,robust=True,cbar_kwargs={'shrink':0.25})
ax.set_title('760 exp(1.3 ratio(865/565)), from Han et al., 2016')
ax = plt.subplot(1, 3,2, projection=proj)
spm2.plot.imshow(cmap=cmap,ax=ax,robust=True,cbar_kwargs={'shrink':0.25})
ax.set_title(' 1.378 exp(4.246 ratio(705/565)), from in situ data')
ax = plt.subplot(1, 3,3, projection=proj)
spm3.plot.imshow(cmap=cmap,ax=ax,robust=True,cbar_kwargs={'shrink':0.25})
ax.set_title(' 6.384 exp(3.215 ratio(705/565)) -57.7, from in situ data')

In [ ]:
a = [610.94*np.pi, 0.2324/np.pi]
Rrs_ = Rrs.isel(wl=3)
spm = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm.name='SPM_N2016'
spm.attrs['units']='mg/L'
spm.attrs['description']='Concentration of suspended particulate matter from band 665 nm'

a = [428.277*np.pi, 0.3051/np.pi]
Rrs_ = Rrs.isel(wl=3)
spm2 = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm2.name='SPM_N2016_han1'
spm2.attrs['units']='mg/L'
spm2.attrs['description']='Concentration of suspended particulate matter from band 665 nm'


a = [1444.853*np.pi, 0.3539/np.pi]
Rrs_ = Rrs.isel(wl=3)
spm3 = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm3.name='SPM_N2016_han2'
spm3.attrs['units']='mg/L'
spm3.attrs['description']='Concentration of suspended particulate matter from band 665 nm'


In [ ]:
plt.figure(figsize=(25,18))
colors = ['mediumblue','cadetblue','teal','gold','orange','chocolate','firebrick']
cmap = mpl.colors.LinearSegmentedColormap.from_list('spm',colors)
fig = plt.figure(figsize=(25, 15))
ax = plt.subplot(1, 3, 1, projection=proj)
spm.plot.imshow(cmap=cmap,ax=ax,robust=True,cbar_kwargs={'shrink':0.25})
ax.set_title('Nechad et al., 2016')
ax = plt.subplot(1, 3,2, projection=proj)
spm2.plot.imshow(cmap=cmap,ax=ax,robust=True,cbar_kwargs={'shrink':0.25})
ax.set_title(' Nechad_Han SAA')
ax = plt.subplot(1, 3,3, projection=proj)
spm3.plot.imshow(cmap=cmap,ax=ax,robust=True,cbar_kwargs={'shrink':0.25})
ax.set_title('Nechad_HAN SAA-H')

### Check blue over green ratio for Chl retrieval with OC2 from NASA
$log_{10}(chlor\_a) = a_0 + \sum\limits_{i=1}^4 a_i \left(log_{10}\left(\frac{R_{rs}(\lambda_{blue})}{R_{rs}(\lambda_{green})}\right)\right)^i$

In [ ]:
Rrs = l2a.l2_prod.Rrs

In [ ]:

# NASA OC2 fro MODIS; bands 488, 547 nm
a = [0.2500,-2.4752,1.4061,-2.8233,0.5405]
# NASA OC2 for OCTS; bands 490, 565 nm
a = [0.2236,-1.8296,1.9094,-2.9481,-0.1718]
# Pelevin et al, 2023, Issyk-Kul
a = [0.1977,-1.8117,1.9742,-2.5635,-0.7218]
ratio = np.log10(Rrs.isel(wl=1)/Rrs.isel(wl=2))
logchl=0
for i in range(len(a)):
    logchl+=a[i]*ratio**i
_chl = 10**(logchl)
_chl.name='Chla_OC2_P2023'
_chl.attrs['units']='mg.m-3'
_chl.attrs['description']= 'Chl-a concentration from NASA OC2 with OCTS parameterization, bands 490, 565 nm',
_chl = _chl.where((_chl >= 0) & (_chl <= 2000))


In [ ]:
plt.figure(figsize=(15,15))
import colorcet as cc
_chl.plot.imshow(cmap=cc.cm.CET_D13,robust=True,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35},vmax=10)

### CDOM retrieval based on Brezonik et al, 2015

In [ ]:
a = [1.872,-0.83]
acdom = np.exp(a[0] + a[1] * np.log(Rrs.isel(wl=1)/Rrs.isel(wl=5)))
acdom.name='a_cdom_b2015'
acdom.attrs['units']='m-1'
acdom.attrs['description']='CDOM absorption at 440 nm-1'
acdom= acdom.where((acdom >= 0) & (acdom <= 60))


In [ ]:
plt.figure(figsize=(15,15))

acdom.plot.imshow(cmap=cc.cm.CET_CBD1,robust=True,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35},vmax=20)

In [ ]:
def cPOC_2(Rrs,p=[2.873,0.945,0.025]):
    ratio1=Rrs.sel(wl=665,method='nearest') / Rrs.sel(wl=490,method='nearest') 
    ratio2=Rrs.sel(wl=665,method='nearest') / Rrs.sel(wl=565,method='nearest') 
    ratio = np.log10(xr.concat([ratio1.assign_coords({'num':1}),ratio1.assign_coords({'num':2})],dim='num').max('num'))
    
    Xpoc = p[0]+p[1]*ratio+p[2]*ratio**2
    return 10**Xpoc

poc = cPOC_2(Rrs)
poc.name = 'cPOC_2'

In [ ]:
plt.figure(figsize=(15,15))

poc.plot.imshow(cmap=cc.cm.blues,robust=True,subplot_kws=dict(projection=proj),cbar_kwargs={'shrink':0.35})

In [ ]:
from obs2co_l2bgen import chl, spm,cdom,transparency


l2a=xr.Dataset({'Rrs':Rrs})
chl_prod= chl(l2a)
chl_prod.process()

# ----------------------
# get SPM parameters
# ----------------------

spm_prod= spm(l2a)
spm_prod.process()

# ----------------------
# get CDOM parameters
# ----------------------

cdom_prod= cdom(l2a)
cdom_prod.process()


# ----------------------
# get transparency parameters
# ----------------------

trans_prod= transparency(l2a)
trans_prod.process()

# ----------------------
# possibity to test other retrievals
# ----------------------
def cPOC_2(Rrs,p=[2.873,0.945,0.025]):
    ''' Ref: Tran, T.K.; Duforêt-Gaurier, L.; Vantrepotte, V.; Jorge, D.S.F.; Mériaux, X.; Cauvin, A.; Fanton d’Andon, O.; Loisel, H.
    Deriving Particulate Organic Carbon in Coastal Waters from Remote Sensing: Inter-Comparison Exercise 
    and Development of a Maximum Band-Ratio Approach. Remote Sens. 2019, 11, 2849. https://doi.org/10.3390/rs11232849 '''
    
    ratio1=Rrs.sel(wl=665,method='nearest') / Rrs.sel(wl=490,method='nearest') 
    ratio2=Rrs.sel(wl=665,method='nearest') / Rrs.sel(wl=565,method='nearest') 
    ratio = np.log10(xr.concat([ratio1.assign_coords({'num':1}),ratio1.assign_coords({'num':2})],dim='num').max('num'))
    
    Xpoc = p[0]+p[1]*ratio+p[2]*ratio**2
    return 10**Xpoc

poc = cPOC_2(Rrs)
poc.name = 'cPOC_2'



# ----------------------
# Merge parameters into xr.Dataset
# ----------------------
l2b = xr.merge([chl_prod.output,spm_prod.output,cdom_prod.output,trans_prod.output,poc,_chl,_spm])



In [ ]:
l2b.to_netcdf('/DATA/vrac/test_l2b.nc')

In [ ]:
import xarray as xr

import grstbx
from grstbx import visual


In [ ]:
#l2b = xr.open_dataset('/DATA/vrac/test_l2b.nc')

vp=visual.view_param(l2b,reproject=True)


In [ ]:
vp.minmax=[0,400]
vp.visu()